## 1. Introduction

This project will explore and analyze the information stored in a particular dataset. In this case, the ACL Anthology dataset (https://aclanthology.org/). We will explore different techniques for obtaining valuable information.

### Task 2: Mining information from Text Data
Using the whole anthologies abstract dataset. Extract a list of the authors and editors per publication and create baskets and perform a search of similar items, for example:

- basket 1: Mostafazadeh Davani Aida,Kiela Douwe,Lambert Mathias,Vidgen, Bertie Prabhakaran Vinodkumar, Waseem, Zeerak
- basket 2: Singh Sumer, Li Sheng

1. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results.

2. For the PCY algorithm, create up to 5 compact hash tables. What is the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

3. Find the final list of k-frequent items (k-tuples) for k=3 and 4. Experiment a bit and describe the best value for the support in each case. Warning: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly (well, basically don't use the naïve approach ;)).

4. Using one of the results of the previous items, for one k (k=2 or 3) find the possible clusters using the 1-NN criteria. Comment your results.

1-NN means that if you have a tuple {A,B,C} and {C,E,F} then because they share one element {C}, then they belong to the same cluster {A,B,C,E,F}.

In [48]:
# import libraries

#from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

import pandas as pd
import numpy
import numpy as np
import os
import re
import binascii
from time import time

from urllib import request
import gzip
import shutil
import time

import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
#from sklearn.decomposition import PCA

import matplotlib

import networkx as nx
import matplotlib.pyplot as plt
import itertools

%matplotlib inline

In [4]:
# Download and extract dataset

url1 = "https://aclanthology.org/anthology+abstracts.bib.gz"
file_name1 = re.split(pattern='/', string=url1)[-1]
r1 = request.urlretrieve(url=url1, filename=file_name1)
txt1 = re.split(pattern=r'\.', string=file_name1)[0] + ".txt"

# Extract it
with gzip.open(file_name1, 'rb') as f_in:
    with open(txt1, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

fname = txt1    

In [5]:
# Various Functions
#==================#

# abstract extracting function
# fname is the file name of the document containing all abstract
# n is the number of abstracts that we will extract
def read_abstracts(fname,n):
    abs = [] #initialize a list variable
    with open(fname, 'r', encoding="utf-8") as f:
        i = 0
        # skip all lines until abstract
        for line in f:
            if "abstract =" in line:
                pattern = '"'
                abstract = re.split(pattern ,line, flags=re.IGNORECASE)[1].split('"')[0]
                if len(abstract)<5: # takes care of empty abstracts
                    pass
                    
                else:
                    abs.append(abstract) # append each abstract to the list
                    i = i + 1
                if i == n:  # number of abstracts to extract
                    return abs
        
        return abs
#=================================================
    
# Shingle function
# k is the number of shingles

def get_shingles(abstract, k):
    """Get all shingles from requested file (hashes of these shingles)
    """
    L = len(abstract)
    shingles = set()  # we use a set to automatically eliminate duplicates
    for i in range(L-k+1):
        shingle = abstract[i:i+k]
        crc = binascii.crc32(shingle.encode('utf-8')) # hash the shingle to a 32-bit integer
        shingles.add(crc)
    return shingles
#=================================================

# jaccard similarity score Function
def jaccard_similarity_score(x, y, errors='ignore'):
    """
    Jaccard Similarity J (A,B) = | Intersection (A,B) | /
                                    | Union (A,B) |
    """
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    if float(union_cardinality) == 0:
        ja = 0
    else:
        ja = intersection_cardinality / float(union_cardinality)
    return ja
#=================================================

# similarity functions
# k is number of shingles and s is the similarity thresholds 
# abstract_list is the list of 1000 abstracts

def similar_items(abstract_list, k, s):
    candidates = []
    #abstract_list = read_abstracts(fname,n)
    for pair in itertools.combinations(abstract_list,2):
        js = jaccard_similarity_score(get_shingles(pair[0], k),get_shingles(pair[1], k))
        
        if js > s:
            #print(pair)
            candidates.append(pair)
            
    return candidates
#=================================================


# fast implementation of Minhashing algorithm
# computes all random hash functions for a shingle at once, using vector operations
# also finds element-wise minimum of two vectors efficiently
def minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = numpy.ones((nsig,)) * (maxShingleID + 1)

    for ShingleID in shingles:
        hashCodes = ((A*ShingleID + B) % nextPrime) % maxShingleID
        numpy.minimum(signature, hashCodes, out=signature)

    return signature
#=================================================

# candidate pair function
def candidate_pair(abstract_list, k, s):
    signatures = []  # signatures for all files
    for abstract in abstract_list:
        shingles = get_shingles(abstract, k)
        signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
        signatures.append(signature)
        
    Nfiles = len(signatures)
    #startTime = time.time()
    candidates = []
    for i in range(Nfiles):
        for j in range(i+1, Nfiles):
            Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
            if Jsim >= s:                                      # two vectors, equivalente to Jaccard
                candidates.append((i,j))
                
            
    return len(candidates)
#=================================================

# Moditied Function for jaccard similarity

def jaccard_similarity_score_mod2(a, b, shingles_list, errors='ignore'): 
    
    sha = shingles_list[a]
    shingles_vector_a = sha
 
    shb = shingles_list[b]
    shingles_vector_b = shb

    jsc = jaccard_similarity_score(shingles_vector_a, shingles_vector_b)
    
    return jsc
#=================================================

# LSH candidates function
def LSH(signatures, bands, rows, Ab, Bb, nextPrime, maxShingleID, s, shingles_list):
    """Locality Sensitive Hashing
    """
    numItems = signatures.shape[1]
    signBands = numpy.array_split(signatures, bands, axis=0) 
    candidates = set()
    for nb in range(bands):
        hashTable = {}
        for ni in range(numItems):
            item = signBands[nb][:,ni]
            hash = (numpy.dot(Ab[nb,:], item) + Bb[nb]) % nextPrime % maxShingleID
            if hash not in hashTable:
                hashTable[hash] = [ni]
            else:
                hashTable[hash].append(ni)
        for _,items in hashTable.items():
            if len(items) > 1:
                L = len(items)
                for i in range(L-1):
                    for j in range(i+1, L):
                        cand = [items[i], items[j]]
                        a = items[i]
                        b = items[j]
                        jsim = jaccard_similarity_score_mod2(a,b, shingles_list) #jaccard similarity function call
                        if jsim >= s:
                            numpy.sort(cand)
                            candidates.add(tuple(cand))
    return candidates
#=================================================

# LSH candidates length function
def LSH_candidates(abstract_list, k, s):
    signatures = []  # signatures for all files
    shingles_list =[]
    for abstract in abstract_list:
        shingles = get_shingles(abstract, k)
        signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
        signatures.append(signature)
        shingles_list.append(shingles)
        
    
    A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows),dtype=numpy.int64)  # now we need a vector of A parameters for each band
    B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ),dtype=numpy.int64)
    signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

  
    candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID, s, shingles_list)
   
    
    return len(candidates)
#=================================================

# New minHash function for Task1 number2 a
def Sim_Method_Property(abstract_list,k,s,bands,rows):
    
    nsig = bands*rows  # hashing function: number of elements in signature, or the number of different random hash functions

    #maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
    #nextPrime = 4294967311  # next prime number after maxShingleID

    #A = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
    #B = numpy.random.randint(0, nextPrime, size=(nsig,),dtype=numpy.int64)
    
    startTime = time.time()
    cand = candidate_pair(abstract_list, k, s)
    execTime = round(((time.time() - startTime)),2)

    minHash_k.append(k)
    minHash_s.append(s)
    Hashing_fn.append(nsig)
    minHash_sim.append(cand)
    minHash_execT.append(execTime)
    
    dict = {'k': minHash_k, 's': minHash_s, 'Hashing_fn': Hashing_fn,'#sim': minHash_sim, 'execTime(sec)': minHash_execT} 
    df = pd.DataFrame(dict)
    
    return df
#=================================================

# New LSH function for Task1 number2 b
def Sim_Method_Property2(abstract_list,k,s,bands,rows):
    
    
    nsig = bands*rows  # hashing function: number of elements in signature, or the number of different random hash functions
    
    startTime = time.time()
    candi = LSH_candidates(abstract_list,k, s)
    execTime = round(((time.time() - startTime)),2)
    
    LSH_k.append(k)
    LSH_s.append(s)
    Hashing_fn2.append(nsig)
    LSH_sim.append(candi)
    LSH_execT.append(execTime)
    
    dict = {'k': LSH_k, 's': LSH_s, 'Hashing_fn': Hashing_fn2,'#sim': LSH_sim, 'execTime(sec)': LSH_execT} 
    df = pd.DataFrame(dict)
    
    return df
#=================================================

# functions for Task1 number 3
# Jaccard distance calculator function

def jacc_dist_calc(abstract_list,k,s,bands,rows):
    
    nsig = bands*rows  # hashing function: number of elements in signature, or the number of different random hash functions
    
    jd_df = candidate_pair_jacc_dist(abstract_list, k, s, nsig)

    return jd_df
#==============

# A modified candidate pair function

def candidate_pair_jacc_dist(abstract_list, k, s, nsig):
    signatures = []  # signatures for all files
    shingles_list = []
    for abstract in abstract_list:
        shingles = get_shingles(abstract, k)
        signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
        signatures.append(signature)
        shingles_list.append(shingles)
        
    Nfiles = len(signatures)
    candidates = []
    jaccard_distance = []
    s_list = []
    #k_list = []
    #h_fn_list = []
    #sign1 = []
    #sign2 = []
    for i in range(Nfiles):
        for j in range(i+1, Nfiles):
            Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in 
            if Jsim >= s:                                      # two vectors, equivalente to Jaccard
                #a = i
                #b = j
                js = jaccard_similarity_score_mod2(i,j, shingles_list)
                jaccard_distance.append(1-js) # jaccard distance calculations
                s_list.append(s)
                #k_list.append(k)
                #h_fn_list.append(nsig)
                candidates.append((i,j))
                #sign1.append(signatures[i])
                #sign2.append(signatures[j])
    
    dict = {'s': s_list, 'candidates': candidates, 'jacc_distance': jaccard_distance} 
    df = pd.DataFrame(dict)
    return df    #len(candidates)
#=================================================
#=================================================
#=================================================
# naive algorithm function for Task2

def naive_items(s):
    start = time.time() # start time
    
    def get_C(k):
        C = {}
        for key in readdata_hands_on(k):  # False report
            if key not in C:
                C[key] = 1
            else:
                C[key] += 1
        #print("Took {}s for k={}".format((time.time() - start), k))
        return C


    C1 = get_C(1)
    C2 = get_C(2)
    
    
     
    L2 = {}
    for key, n in C2.items():
        if n >= s:
            L2[key] = n
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#=================================================

# a-priori algorithm function for Task2

def apriori_items(s):
    
    start = time.time() # start time
    N = s
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) 
    #print(len(C2_items))
    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on(k=2):
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))
    
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('A-priori: {} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#=================================================

def readdata_hands_on(k, fname='anthology_authors.csv'):
    
    with open(fname, "rt", encoding='latin1') as f:
        for line in f:
            C_k  = line.rstrip().split(';')
            for itemset in itertools.combinations(C_k, k):
                    yield frozenset(itemset)
#=================================================

# pcy algorithm function

def pcy_items(s):
    
    start = time.time() # start time
    N = s # frequency threshold
    
    # hash table
    max_hash1 = 10 * 1000000
    H1 = np.zeros((max_hash1, ), dtype=np.int)

    for key in readdata_hands_on(k=2):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()])
    
    #print(len(C2_items))
    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if H1[hash_cell_1] < N:
            continue

        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))
    
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('{} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return L2
#=================================================

def readdata_hands_on2(k,report=False,fname='anthology_authors.csv'):
    
    with open(fname, "rt", encoding='latin1') as f:
        for line in f:
            C_k  = line.rstrip().split(';')
            for itemset in itertools.combinations(C_k, k):
                    yield frozenset(itemset) 
#=================================================
                    
# Task 2 hash tables functions

##### PCY Algorithm

# One compact hash table 
# PCY algorithm function

def oneHashTablePCY(s):
    
    start = time.time() # start time
    N = s # frequency threshold
    
    # hash table
    max_hash1 = 10 * 1000000
    H1 = np.zeros((max_hash1, ), dtype=np.int)

    for key in readdata_hands_on2(k=2, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        
        
    #### === one compact hash table ====####
    H_good_1 = set(np.where(H1 >= N)[0])
    del H1
    
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on2(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()])

    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on2(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if hash_cell_1 not in H_good_1:
            continue

        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))

   
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('{} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#==============

# Two compact hash table 
# PCY algorithm function

def twoHashTablePCY(s):
    
    start = time.time() # start time
    N = s # frequency threshold
    
    # hash tables
    max_hash1 = 5*1000000-673
    max_hash2 = 5*1000000+673
    
    H1 = np.zeros((max_hash1,), dtype=np.int)
    H2 = np.zeros((max_hash2,), dtype=np.int)

    for key in readdata_hands_on2(k=2, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        hash_cell_2 = hash(key) % max_hash2
        H2[hash_cell_2] += 1
        
    #### === two compact hash tables ====####
    H_good_1 = set(np.where(H1 >= N)[0])
    H_good_2 = set(np.where(H2 >= N)[0])
    del H1
    del H2
    
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on2(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()])

    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on2(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if hash_cell_1 not in H_good_1:
            continue
        
        hash_cell_2 = hash(key) % max_hash2
        if hash_cell_2 not in H_good_2:
            continue
                

        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))

   
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('{} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#==============

# Three compact hash table 
# PCY algorithm function

def threeHashTablePCY(s):
    
    start = time.time() # start time
    N = s # frequency threshold
    
    # hash tables
    max_hash1 = 5*1000000-673
    max_hash2 = 5*1000000+673
    max_hash3 = 5*1000000-128
    
    H1 = np.zeros((max_hash1,), dtype=np.int)
    H2 = np.zeros((max_hash2,), dtype=np.int)
    H3 = np.zeros((max_hash3,), dtype=np.int)

    for key in readdata_hands_on2(k=2, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        hash_cell_2 = hash(key) % max_hash2
        H2[hash_cell_2] += 1
        hash_cell_3 = hash(key) % max_hash3
        H3[hash_cell_3] += 1
        
        
    #### === three compact hash tables ====####
    H_good_1 = set(np.where(H1 >= N)[0])
    H_good_2 = set(np.where(H2 >= N)[0])
    H_good_3 = set(np.where(H3 >= N)[0])
    del H1
    del H2
    del H3
    
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on2(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()])

    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on2(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if hash_cell_1 not in H_good_1:
            continue
        
        hash_cell_2 = hash(key) % max_hash2
        if hash_cell_2 not in H_good_2:
            continue
         
        hash_cell_3 = hash(key) % max_hash3
        if hash_cell_3 not in H_good_3:
            continue

        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))

   
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('{} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#==============

# Four compact hash table 
# PCY algorithm function

def fourHashTablePCY(s):
    
    start = time.time() # start time
    N = s # frequency threshold
    
    # hash tables
    max_hash1 = 5*1000000-673
    max_hash2 = 5*1000000+673
    max_hash3 = 5*1000000-128
    max_hash4 = 5*1000000+128
    
    H1 = np.zeros((max_hash1,), dtype=np.int)
    H2 = np.zeros((max_hash2,), dtype=np.int)
    H3 = np.zeros((max_hash3,), dtype=np.int)
    H4 = np.zeros((max_hash4,), dtype=np.int)

    for key in readdata_hands_on2(k=2, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        hash_cell_2 = hash(key) % max_hash2
        H2[hash_cell_2] += 1
        hash_cell_3 = hash(key) % max_hash3
        H3[hash_cell_3] += 1
        hash_cell_4 = hash(key) % max_hash4
        H4[hash_cell_4] += 1
        
    #### === four compact hash tables ====####
    H_good_1 = set(np.where(H1 >= N)[0])
    H_good_2 = set(np.where(H2 >= N)[0])
    H_good_3 = set(np.where(H3 >= N)[0])
    H_good_4 = set(np.where(H4 >= N)[0])
    del H1
    del H2
    del H3
    del H4
    
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on2(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()])

    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on2(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if hash_cell_1 not in H_good_1:
            continue
        
        hash_cell_2 = hash(key) % max_hash2
        if hash_cell_2 not in H_good_2:
            continue
         
        hash_cell_3 = hash(key) % max_hash3
        if hash_cell_3 not in H_good_3:
            continue

        hash_cell_4 = hash(key) % max_hash4
        if hash_cell_4 not in H_good_4:
            continue
        
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))

   
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('{} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#==============

# Five compact hash table 
# PCY algorithm function

def fiveHashTablePCY(s):
    
    start = time.time() # start time
    N = s # frequency threshold
    
    # hash tables
    max_hash1 = 5*1000000-673
    max_hash2 = 5*1000000+673
    max_hash3 = 5*1000000-128
    max_hash4 = 5*1000000+128
    max_hash5 = 5*1000000+256
    
    H1 = np.zeros((max_hash1,), dtype=np.int)
    H2 = np.zeros((max_hash2,), dtype=np.int)
    H3 = np.zeros((max_hash3,), dtype=np.int)
    H4 = np.zeros((max_hash4,), dtype=np.int)
    H5 = np.zeros((max_hash5,), dtype=np.int)

    for key in readdata_hands_on2(k=2, report=False):
        hash_cell_1 = hash(key) % max_hash1
        H1[hash_cell_1] += 1
        hash_cell_2 = hash(key) % max_hash2
        H2[hash_cell_2] += 1
        hash_cell_3 = hash(key) % max_hash3
        H3[hash_cell_3] += 1
        hash_cell_4 = hash(key) % max_hash4
        H4[hash_cell_4] += 1
        hash_cell_5 = hash(key) % max_hash5
        H5[hash_cell_5] += 1
        
    #### === five compact hash tables ====####
    H_good_1 = set(np.where(H1 >= N)[0])
    H_good_2 = set(np.where(H2 >= N)[0])
    H_good_3 = set(np.where(H3 >= N)[0])
    H_good_4 = set(np.where(H4 >= N)[0])
    H_good_5 = set(np.where(H5 >= N)[0])
    del H1
    del H2
    del H3
    del H4
    del H5
    
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on2(k=1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()])

    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on2(k=2):
        # hash-based filtering stage from PCY
        hash_cell_1 = hash(key) % max_hash1
        if hash_cell_1 not in H_good_1:
            continue
        
        hash_cell_2 = hash(key) % max_hash2
        if hash_cell_2 not in H_good_2:
            continue
         
        hash_cell_3 = hash(key) % max_hash3
        if hash_cell_3 not in H_good_3:
            continue

        hash_cell_4 = hash(key) % max_hash4
        if hash_cell_4 not in H_good_4:
            continue
        
        hash_cell_5 = hash(key) % max_hash5
        if hash_cell_5 not in H_good_5:
            continue
        
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))

   
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('{} items with >{} occurances'.format(len(L2), N))
    
    # append results
    s_list.append(s)
    L2_counts_list.append(len(L2))
    Wall_time_list_ns.append(time.time() - start)
    return
#=================================================

# Lets modify our A-priori function for frequent 3 tuple 

def apriori_3tuple(s,k1,k2,k3):
    
    start = time.time() # start time
    N = s
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on(k1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) 
    #print(len(C2_items))
    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on(k2):
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))
    
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('A-priori: {} items with >{} occurances'.format(len(L2), N))
    
    
    L1_items = set(L1.keys())
    L2_items = set(L2.keys())
    
    # SLOW! (too many possible 3-tuples) So let's be smart and use some time constrain.
    start = time.time()
    PERIOD_OF_TIME = 10

    # find frequent 3-tuples
    C3 = {}
    for key in readdata_hands_on2(k3):
        
        # filter out non-frequent tuples
        # A-Priori filtering, option 2: generate all possible subsets and check that they all are frequent
        non_freq_1 = set([frozenset(x) for x in itertools.combinations(list(key), 1)]) - L1_items
        if len(non_freq_1) > 0:
            continue

        non_freq_2 = set([frozenset(x) for x in itertools.combinations(list(key), 2)]) - L2_items
        if len(non_freq_2) > 0:
            continue

        # record frequent tuples
        if key not in C3:
            C3[key] = 1
        else:
            C3[key] += 1
    
        ################################
        # break out of a slow function #
        if time.time() > start + PERIOD_OF_TIME : 
            print('Time is running out')
            break
        
    #print("{} items".format(len(C3)))
    
    # filter stage
    L3 = {}
    for key, count in C3.items():
        if count >= N:
            L3[key] = count
    #print('{} items with >{} occurances'.format(len(L3), Ntest))
    
    # append results
    s_list.append(s)
    L3_counts_list.append(len(L3))
    Wall_time_list_ns.append(time.time() - start)
    
    
    return
#=================================================

# Lets modify our A-priori function for frequent 4 tuple

def apriori_4tuple(s,k1,k2,k3,k4):
    
    start = time.time() # start time
    N = s
    # find frequent individual items
    C1 = {}
    for key in readdata_hands_on(k1):
        if key not in C1:
            C1[key] = 1
        else:
            C1[key] += 1    
        
    #print("{} items".format(len(C1)))
    
    # filter stage
    L1 = {}
    for key, count in C1.items():
        if count >= N:
            L1[key] = count
    #print('{} items with >{} occurances'.format(len(L1), N))
    
    # List comprehensions in python
    C2_items = set([a.union(b) for a in L1.keys() for b in L1.keys()]) 
    #print(len(C2_items))
    
    # find frequent 2-tuples
    C2 = {}
    for key in readdata_hands_on(k2):
        # filter out non-frequent tuples
        if key not in C2_items:
            continue

        # record frequent tuples
        if key not in C2:
            C2[key] = 1
        else:
            C2[key] += 1
        
    #print("{} items".format(len(C2)))
    
    # filter stage
    L2 = {}
    for key, count in C2.items():
        if count >= N:
            L2[key] = count
    #print('A-priori: {} items with >{} occurances'.format(len(L2), N))
    
    
    L1_items = set(L1.keys())
    L2_items = set(L2.keys())
    
    # SLOW! (too many possible 3-tuples) So let's be smart and use some time constrain.
    start = time.time()
    PERIOD_OF_TIME = 10

    # find frequent 3-tuples
    C3 = {}
    for key in readdata_hands_on2(k3):
        
        # filter out non-frequent tuples
        # A-Priori filtering, option 2: generate all possible subsets and check that they all are frequent
        non_freq_1 = set([frozenset(x) for x in itertools.combinations(list(key), 1)]) - L1_items
        if len(non_freq_1) > 0:
            continue

        non_freq_2 = set([frozenset(x) for x in itertools.combinations(list(key), 2)]) - L2_items
        if len(non_freq_2) > 0:
            continue

        # record frequent tuples
        if key not in C3:
            C3[key] = 1
        else:
            C3[key] += 1
    
        ################################
        # break out of a slow function #
        if time.time() > start + PERIOD_OF_TIME : 
            print('Time is running out for freq 3-tuple')
            break
        
    #print("{} items".format(len(C3)))
    
    # filter stage
    L3 = {}
    for key, count in C3.items():
        if count >= N:
            L3[key] = count
    #print('{} items with >{} occurances'.format(len(L3), Ntest))
    
    
    # find frequent 4 tuple ==================================
    L3_items = set(L3.keys())
    
    # SLOW! (too many possible 4-tuples) So let's be smart and use some time constrain.
    start = time.time()
    PERIOD_OF_TIME = 10  

    # find frequent 4-tuples
    C4 = {}
    for key in readdata_hands_on2(k4):
        
        # filter out non-frequent tuples
        # A-Priori filtering, option 2: generate all possible subsets and check that they all are frequent
        non_freq_1 = set([frozenset(x) for x in itertools.combinations(list(key), 1)]) - L1_items
        if len(non_freq_1) > 0:
            continue

        non_freq_2 = set([frozenset(x) for x in itertools.combinations(list(key), 2)]) - L2_items
        if len(non_freq_2) > 0:
            continue

        non_freq_3 = set([frozenset(x) for x in itertools.combinations(list(key), 3)]) - L3_items
        if len(non_freq_3) > 0:
            continue   
            
            
        # record frequent tuples
        if key not in C4:
            C4[key] = 1
        else:
            C4[key] += 1
    
        ################################
        # break out of a slow function #
        if time.time() > start + PERIOD_OF_TIME : 
            print('Time is running out for freq 4-tuple')
            break
        
    #print("{} items".format(len(C4)))
    
    # filter stage
    L4 = {}
    for key, count in C4.items():
        if count >= N:
            L4[key] = count
    #print('{} items with >{} occurances'.format(len(L4), N))
    

    
    # append results
    s_list.append(s)
    L4_counts_list.append(len(L4))
    Wall_time_list_ns.append(time.time() - start)
    
    
    return 
#=================================================


1. Find the frequent pair of items (2-tuples) using the naïve, A-priori and PCY algorithms. For each of these compare the time of execution and results for supports s=10, 50, 100. Comment your results.


In [6]:
r1[0] #this is the bib compressed file name

'anthology+abstracts.bib.gz'

In [5]:
# unpacking our already downloaded anthology+abstracts.bib.gz file
import gzip 
import shutil
with gzip.open(r1[0], 'rb') as f_in:
    with open('anthology.bib', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [9]:
#!pip install bibtexparser  # install this is not available

     |████████████████████████████████| 46 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 2.7 MB/s eta 0:00:01�███████████████████▊      | 665 kB 2.7 MB/s eta 0:00:01
  Created wheel for bibtexparser: filename=bibtexparser-1.2.0-py3-none-any.whl size=36712 sha256=2231d9cfd623932f56b4943114d990ff760f85298d8e5e0a51345ebd01b09c9e
  Stored in directory: /home/jovyan/.cache/pip/wheels/3e/13/1d/09c37a40f39ddd7b226719a797f1896a5b95d730de27e7a505
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=849438fdd4546e8f3ce6cc877ce49f715b081e3d08e27084ff6071dfb8bfb7ca
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built bibtexparser future


In [10]:
# Extract the authors and editors names
# This cell takes about 10 - 13mins to extract

import bibtexparser

with open('anthology.bib', encoding="utf-8") as bibtex_file:
    bib_database = bibtexparser.bparser.BibTexParser(common_strings=True).parse_file(bibtex_file)


bib_df = pd.DataFrame(bib_database.entries)

author_editor = bib_df[['author', 'editor']]

In [11]:
author_editor.head() #sample printout

,author,editor
0,NaN,"Mostafazadeh Davani, Aida and\nKiela, Douwe ..."
1,"Singh, Sumer and\nLi, Sheng",NaN
2,"Hahn, Vanessa and\nRuiter, Dana and\nKleinba...",NaN
3,"Caselli, Tommaso and\nBasile, Valerio and\nM...",NaN
4,"Kirk, Hannah and\nJun, Yennie and\nRauba, Pa...",NaN


In [12]:
### Lets pre-process

# make a copy of it
aut_edit = author_editor.copy() 

# create a new column
aut_edit['author_editor'] = 'a'


# populate this new column with values from the original columns
for i in range(len(aut_edit)):
    if pd.isna(aut_edit.iloc[i,0]):
        aut_edit.iloc[i,2] = aut_edit.iloc[i,1]
        
    else:
        aut_edit.iloc[i,2] = aut_edit.iloc[i,0]
        

        
# drop old columns
aut_edit.drop(labels=['author','editor'], axis=1, inplace=True)

# drop na
aut_edit = aut_edit.dropna()


# Replace and\n with ; (which separates each author's/editor's full name)
aut_edit['author_editor'] = aut_edit['author_editor'].str.replace('and\n',';')


# see the new head
aut_edit.head()

,author_editor
0,"Mostafazadeh Davani, Aida ;Kiela, Douwe ;Lam..."
1,"Singh, Sumer ;Li, Sheng"
2,"Hahn, Vanessa ;Ruiter, Dana ;Kleinbauer, Tho..."
3,"Caselli, Tommaso ;Basile, Valerio ;Mitrovi{\..."
4,"Kirk, Hannah ;Jun, Yennie ;Rauba, Paulius ;..."


In [13]:
# save a copy 

aut_edit.to_csv('anthology_authors.csv')

#### Finding the frequent pair of items (2-tuples) using the naïve algorithm

In [7]:
aut_edit = pd.read_csv('anthology_authors.csv')

In [9]:
nbaskets = aut_edit.shape[0]
nbaskets

73197

In [10]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

In [11]:
####
###### for supports s=10

s = 10
naive_items(s)

In [12]:
####
###### for supports s=50

s = 50
naive_items(s)

In [13]:
####
###### for supports s=100

s = 100
naive_items(s)

In [14]:
# naive algorithm result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
naive_algo = pd.DataFrame(dict)

#### Finding the frequent pair of items (2-tuples) using the A-priori  algorithm

In [15]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

In [16]:
####
###### for supports s=10

s = 10
apriori_items(s)

In [17]:
####
###### for supports s=50

s = 50
apriori_items(s)

In [18]:
####
###### for supports s=100

s = 100
apriori_items(s)

In [19]:
# apriori algorithm result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
apriori_algo = pd.DataFrame(dict)

#### Finding the frequent pair of items (2-tuples) using the PCY algorithm

In [20]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

In [50]:
####
###### for supports s=10
###### use zzz to collect the un-needed output(the print out was so much)
s = 10
zzz = pcy_items(s)

In [22]:
####
###### for supports s=50

s = 50
pcy_items(s)

{frozenset({'Sumita, Eiichiro"', 'Utiyama, Masao  '}): 56,
 frozenset({'Bhattacharyya, Pushpak"', 'Ekbal, Asif  '}): 57}

In [23]:
####
###### for supports s=100

s = 100
pcy_items(s)

{}

In [24]:
# pcy algorithm result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_algo = pd.DataFrame(dict)

##### Number1 - Results

In [25]:

print('Naive Algorithm results'); print(naive_algo); print(''); print('A-priori Algorithm results'); print(apriori_algo); print(''); print('PCY Algorithm results'); print(pcy_algo)


Naive Algorithm results
     s  lenL2  wall_time(s)
0   10    428      1.963969
1   50      2      1.730492
2  100      0      1.516440

A-priori Algorithm results
     s  lenL2  wall_time(s)
0   10    428     17.699204
1   50      2      0.678411
2  100      0      0.627381

PCY Algorithm results
     s  lenL2  wall_time(s)
0   10    428     19.636965
1   50      2      1.558505
2  100      0      1.460125


##### COMMENT
- They all had the same number of items occurences greater than each support thresholds
- However for support threshold 10, naive algorithm was the fastest while pcy was the slowest.
- For higher support thresholds the speed of execution of apriori and pcy were faster than that of naive algorithm; but in this instance, apriori algorithm carried the day as per speed. If we were to factor in memory usage, pcy will be the best

2. For the PCY algorithm, create up to 5 compact hash tables. What is the difference in results and time of execution for 1,2,3,4 and 5 tables? Comment your results.

In [26]:
slist = [10, 50, 100] # various support thresholds

In [27]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

#looping over the various support threshholds
for x in range(len(slist)):
    oneHashTablePCY(slist[x])

#  pcy 1 compact hash table result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_1HashTable = pd.DataFrame(dict)

In [28]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

#looping over the various support threshholds
for x in range(len(slist)):
    twoHashTablePCY(slist[x])

#  pcy 2 compact hash tables result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_2HashTable = pd.DataFrame(dict)

In [29]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

#looping over the various support threshholds
for x in range(len(slist)):
    threeHashTablePCY(slist[x])

# pcy 3 compact hash tables result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_3HashTable = pd.DataFrame(dict)

In [30]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

#looping over the various support threshholds
for x in range(len(slist)):
    fourHashTablePCY(slist[x])

# pcy 4 compact hash tables result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_4HashTable = pd.DataFrame(dict)

In [31]:
# initializing out list 
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

#looping over the various support threshholds
for x in range(len(slist)):
    fiveHashTablePCY(slist[x])

# pcy 5 compact hash tables result
dict = {'s': s_list, 'lenL2': L2_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_5HashTable = pd.DataFrame(dict)

##### Number 2 - Results

In [32]:

print('pcy 1 compact hash tables result'); print(pcy_1HashTable);print('');
print('pcy 2 compact hash tables result'); print(pcy_2HashTable);print('');
print('pcy 3 compact hash tables result'); print(pcy_3HashTable);print('');
print('pcy 4 compact hash tables result'); print(pcy_4HashTable);print('');
print('pcy 5 compact hash tables result'); print(pcy_5HashTable)


pcy 1 compact hash tables result
     s  lenL2  wall_time(s)
0   10    428     17.723017
1   50      2      1.241099
2  100      0      1.202283

pcy 2 compact hash tables result
     s  lenL2  wall_time(s)
0   10    428     16.029233
1   50      2      1.519577
2  100      0      1.772187

pcy 3 compact hash tables result
     s  lenL2  wall_time(s)
0   10    428     16.863230
1   50      2      1.930255
2  100      0      1.855751

pcy 4 compact hash tables result
     s  lenL2  wall_time(s)
0   10    428     19.034633
1   50      2      2.294307
2  100      0      2.186956

pcy 5 compact hash tables result
     s  lenL2  wall_time(s)
0   10    428     20.163187
1   50      2      2.606501
2  100      0      2.651404


##### COMMENT
- The execution times of support threshold 10 as the compact hashing tables increases, overall, seems to be fairly consistent
- But the other support thresholds shows clearly that the times are increasing, showing that more hashing is been done which should lead to a better memory management.

3. Find the final list of k-frequent items (k-tuples) for k=3 and 4. Experiment a bit and describe the best value for the support in each case. Warning: You can use any of the three algorithms, but be careful, because the algorithm can take too long if you don't chose it properly (well, basically don't use the naïve approach ;)).

The for loop cells will take some time to spin up

In [33]:
# initializing out list 
s_list = []
#L2_counts_list = []
L3_counts_list = []
Wall_time_list_ns = []

In [34]:
#looping over the various support threshholds


for x in range(len(slist)):
    apriori_3tuple(slist[x],k1=1, k2=2, k3=3)

# result
dict = {'s': s_list, 'lenL3': L3_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_3tuple = pd.DataFrame(dict)

In [35]:
# initializing out list 
s_list = []
L4_counts_list = []
Wall_time_list_ns = []

In [36]:
#looping over the various support threshholds

slist2 = [8,9]

for x in range(len(slist2)):
    apriori_4tuple(slist2[x],k1=1, k2=2, k3=3, k4=4)

# result
dict = {'s': s_list, 'lenL4': L4_counts_list, 'wall_time(s)': Wall_time_list_ns} 
pcy_4tuple = pd.DataFrame(dict)

Time is running out for freq 4-tuple
Time is running out for freq 4-tuple


##### Number 3 - Results

In [37]:
print('3-tuple results'); print(pcy_3tuple);print('');
print('4-tuple results'); print(pcy_4tuple)

3-tuple results
     s  lenL3  wall_time(s)
0   10    109      2.315736
1   50      0      2.405657
2  100      0      2.130113

4-tuple results
   s  lenL4  wall_time(s)
0  8     15     10.105361
1  9     38     12.598465


##### COMMENT
- 3-tuples started appearing for s values from 10
- And the 4-tuples started appearing for s values 9 and below
- Notable is the time, 3-tuples operations were faster, because the hashing tables were less

4. Using one of the results of the previous items, for one k (k=2 or 3) find the possible clusters using the 1-NN criteria. Comment your results.

1-NN means that if you have a tuple {A,B,C} and {C,E,F} then because they share one element {C}, then they belong to the same cluster {A,B,C,E,F}.

In [38]:
# we are dealing here with K=1-Nearest Neighbor

# We need to collect the L2 results from our support threshold s, 10
# and 50 using our earlier function for 2-tuple pcy
s_list = []
L2_counts_list = []
Wall_time_list_ns = []

L2_s10 = pcy_items(10)
L2_s50 = pcy_items(50)

In [39]:
#converted frozensets to list
L2_s10_list = []
L2_s10_list.append([list(x) for x in L2_s10])

In [40]:
# frozensets converted to list, we print 5 out of 426
L2_s10_list[0][:5]

[['Utiyama, Masao  ', 'Sumita, Eiichiro"'],
 ['Sumita, Eiichiro"', 'Watanabe, Taro  '],
 ['Federmann, Christian  ', 'Chatterjee, Rajen  '],
 ['Graham, Yvette  ', 'Chatterjee, Rajen  '],
 ['Haddow, Barry  ', 'Chatterjee, Rajen  ']]

In [41]:
#grp1 = []
#i = 0
#for x in range(len(L2_s10_list[0])):
#    if i == 20:
#        break
        
#    else:
        
#        count = []
#        names = []
#        for y in range(len(L2_s10_list[0]) - 1):
#            for item in range(len(L2_s10_list[0][y])):
#                if (L2_s10_list[0][x][0]) == (L2_s10_list[0][y][item]):
#                    count.append(y)
#                    #names1.append(L2_s10_list[0][y])
#            print(count)
#            grp1.append(count)
#            #print(names)
#            #i += 1
        

###=================== Extracting the Clusters
### I tested with 20 samples as in the above, before applying to the whole dataset

grp1 = []
i = 0
for x in range(len(L2_s10_list[0])):  
    count = []
    names = []
    for y in range(len(L2_s10_list[0]) - 1):
        for item in range(len(L2_s10_list[0][y])):
            if (L2_s10_list[0][x][0]) == (L2_s10_list[0][y][item]):
                count.append(y)
                #names1.append(L2_s10_list[0][y])
        #print(count)
        grp1.append(count)
        #print(names)
        #i += 1
        

In [42]:
# This filters the list and remove duplicates (this process took a bit of time)

final_grp = [i for j, i in enumerate(grp1) if i not in grp1[:j]] 


In [43]:
final_grp[:18] #Lets see the first 18 items out of 289

[[0, 35, 117, 118, 352],
 [0, 1, 353],
 [2, 8, 9, 10, 11, 12, 24, 25, 45],
 [3, 8, 13, 14, 15, 26],
 [4, 9, 13, 16, 17, 18, 27, 28, 252, 276, 306],
 [5, 10, 14, 16, 19, 20, 29, 30, 253],
 [6, 11, 17, 19, 21],
 [7, 12, 15, 18, 20, 21, 31, 32, 254, 342],
 [22, 24, 26, 27, 29, 31, 33, 255],
 [23, 25, 28, 30, 32, 33, 34, 63, 291, 304],
 [34],
 [36, 39, 160],
 [36, 37, 167],
 [38, 158, 159, 182],
 [40],
 [41],
 [42],
 [43]]

In [44]:
# This for loop counts and prints out the clusters(if both print
# statements are uncommented)

clusters_names = []
clusters_names_counter = 0
for i in range(len(final_grp)):
    if len(final_grp[i]) > 1: # this makes such the cluster must be at least 2 data points
        for j in range(len(final_grp[i])):
            clusters_names.append(L2_s10_list[0][final_grp[i][j]])
        #print(L2_s10_list[0][final_grp[i][j]])

        #print("")  
        clusters_names_counter += 1

##### Number 4 - Results

In [45]:
# print out cluster number
print ("There are {} clusters".format(clusters_names_counter))


There are 121 clusters


In [46]:
# Let print out the first 5 clusters
i = 0
clusters_names = []
clusters_names_counter = 0
for i in range(len(final_grp)):
    for j in range(len(final_grp[i])):
        clusters_names.append(L2_s10_list[0][final_grp[i][j]])
        print(L2_s10_list[0][final_grp[i][j]])

    print("")  
    clusters_names_counter += 1
    i += 1
    if i == 5:
        break

['Utiyama, Masao  ', 'Sumita, Eiichiro"']
['Utiyama, Masao  ', 'Sumita, Eiichiro  ']
['Utiyama, Masao  ', 'Wang, Rui  ']
['Chen, Kehai  ', 'Utiyama, Masao  ']
['Finch, Andrew  ', 'Utiyama, Masao  ']

['Utiyama, Masao  ', 'Sumita, Eiichiro"']
['Sumita, Eiichiro"', 'Watanabe, Taro  ']
['Finch, Andrew  ', 'Sumita, Eiichiro"']

['Federmann, Christian  ', 'Chatterjee, Rajen  ']
['Graham, Yvette  ', 'Federmann, Christian  ']
['Federmann, Christian  ', 'Haddow, Barry  ']
['Federmann, Christian  ', 'Huck, Matthias  ']
['Yepes, Antonio Jimeno  ', 'Federmann, Christian  ']
['Koehn, Philipp  ', 'Federmann, Christian  ']
['Monz, Christof  ', 'Federmann, Christian  ']
['Federmann, Christian  ', 'Negri, Matteo  ']
['Post, Matt  ', 'Federmann, Christian  ']

['Graham, Yvette  ', 'Chatterjee, Rajen  ']
['Graham, Yvette  ', 'Federmann, Christian  ']
['Graham, Yvette  ', 'Haddow, Barry  ']
['Graham, Yvette  ', 'Huck, Matthias  ']
['Koehn, Philipp  ', 'Graham, Yvette  ']
['Graham, Yvette  ', 'Monz, Chris

COMMENTS
- We had about 121 clusters after filtering out some disparsed node, which made it 289 (within these cluster there could be major or massive ones that other methods could build categories as cluster1, cluster2 and so on)
- 1-NN yielded clusters with upto 11 memebers(and could even be more if sorted), this method is more effective and clearer than the one used for the abstracts